# Algoritmo de recomendação

Neste notebook, o nosso intuito será criar um algoritmo de recomendação que se baseie nas interações de um usuário com diversos objetos (content-based recommendation).

Para isso, utilizaremos um dataset simples composto por mais de 5 mil jogos presentes na Steam e quantas horas cada usuário passou jogando aquele título. 
Em termos gerais, o objetivo é utilizar o tempo de jogo para recomendar outros títulos para o usuário.

O dataset pode ser acessado em: https://www.kaggle.com/datasets/tamber/steam-video-games

## Método

1. Realizar a limpeza e organização do dataset
2. Análise e substituição das features categóricas para numéricas
3. Criação de uma matriz de relação
4. Cálculo de similaridades
5. Recomendação (voilà!)

# Organizando o dataset

Iniciaremos realizando os imports necessários de bibliotecas

In [68]:
import pandas as pd
import numpy as np

In [69]:
data = pd.read_csv(r"C:\Users\abril\OneDrive\Documentos\Turing\Portfólio\Algoritmo de Recomendação\steam-200k.csv")
data.drop(columns = ('0'), inplace = True)
data.rename(columns = {"151603712": 'user_id', "The Elder Scrolls V Skyrim": "title", "purchase":'situation', '1.0': 'time' }, inplace = True)
data.head()

,user_id,title,situation,time
0,151603712,The Elder Scrolls V Skyrim,play,273.0
1,151603712,Fallout 4,purchase,1.0
2,151603712,Fallout 4,play,87.0
3,151603712,Spore,purchase,1.0
4,151603712,Spore,play,14.9


Numa breve inspeção do dataset, é  possível visualizar a sua organização (ou tentativa de organização). Cada usuário tem uma série de títulos associados ao seu ID, assim como o tempo jogado e a situação daquele título, se foi jogado ou apenas comprado. Em geral, títulos que apenas foram comprados constam como "purchase" em situation e apresentam 1.0 na coluna time.

Para melhor clareza, optamos por substituir a coluna situation por played, que assume o valor 1 quando o usuário já jogou aquele título e 0 quando ainda não.

In [70]:
#not_played = data[data['time'] == 1.0]['time']
data['played'] = data[data['time'] == 1.0]['time']
data['played'].replace(to_replace = 1.0, value = 0, inplace = True)
data['played'].fillna(value = 1, inplace = True)
situation = data['situation']
data.drop(columns = 'situation', inplace = True)
data['time'].replace(to_replace = 1.0, value = 0.0, inplace = True)
data.head()

,user_id,title,time,played
0,151603712,The Elder Scrolls V Skyrim,273.0,1.0
1,151603712,Fallout 4,0.0,0.0
2,151603712,Fallout 4,87.0,1.0
3,151603712,Spore,0.0,0.0
4,151603712,Spore,14.9,1.0


In [71]:
data.user_id.unique().shape[0]

12393

In [72]:
data.title.unique().shape[0]

5155

O dataset possui 5155 títulos únicos e 12393 usuários registrados

# Introduzindo features numéricas

Para aproveitar a unicidade de cada usuário de maneira mais inteligente visando facilitar o trabalho de construção do algoritmo, substituíremos cada user_id por um número correspondente. Sendo assim, os usuários passaram ser registrados como 1, 2, 3, ..., 12393. 
Faremos isso com o uso de um dicionário que armazena a posição de cada usuário em um vetor.

In [73]:
dicUsers = {}

for i, k in enumerate(data['user_id'].unique()):
    dicUsers[k] = i

In [74]:
print(dicUsers)

{151603712: 0, 187131847: 1, 59945701: 2, 53875128: 3, 234941318: 4, 140954425: 5, 26122540: 6, 176410694: 7, 197278511: 8, 150128162: 9, 197455089: 10, 63024728: 11, 297811211: 12, 76933274: 13, 218323237: 14, 302186258: 15, 126340495: 16, 256193015: 17, 194895541: 18, 30007387: 19, 170625356: 20, 159538705: 21, 167362888: 22, 208649703: 23, 299889828: 24, 225987202: 25, 195071563: 26, 254906420: 27, 247160953: 28, 308653033: 29, 144138643: 30, 197902002: 31, 97298878: 32, 173909336: 33, 198572546: 34, 219509107: 35, 202906503: 36, 92107940: 37, 251431515: 38, 233558010: 39, 99189757: 40, 30695285: 41, 259648553: 42, 201069271: 43, 48845802: 44, 226212066: 45, 221430493: 46, 62923086: 47, 250006052: 48, 65117175: 49, 227944885: 50, 144004384: 51, 236557903: 52, 11373749: 53, 140293612: 54, 187851224: 55, 192921532: 56, 54103616: 57, 222277839: 58, 298547051: 59, 264253640: 60, 125718844: 61, 230599183: 62, 280061602: 63, 38763767: 64, 164543231: 65, 211277578: 66, 214167822: 67, 16361

In [75]:
data['user_id'].replace(to_replace = dicUsers, inplace = True)

In [76]:
names = ((data.title.unique()))

In [77]:
title_id = np.arange(0, 5155)
print(title_id)

[   0    1    2 ... 5152 5153 5154]


Ainda pensando nas operações de organização futuras, substituiremos o nome de cada jogo por um número correspondente. Utiizaremos um método muito similar aquele utilizado para substituir o user_id

In [78]:
dic = {}
for i, l in enumerate(data['title'].unique()):
    dic[l] = i 

In [79]:
print(dic)

{'The Elder Scrolls V Skyrim': 0, 'Fallout 4': 1, 'Spore': 2, 'Fallout New Vegas': 3, 'Left 4 Dead 2': 4, 'HuniePop': 5, 'Path of Exile': 6, 'Poly Bridge': 7, 'Left 4 Dead': 8, 'Team Fortress 2': 9, 'Tomb Raider': 10, 'The Banner Saga': 11, 'Dead Island Epidemic': 12, 'BioShock Infinite': 13, 'Dragon Age Origins - Ultimate Edition': 14, 'Fallout 3 - Game of the Year Edition': 15, 'SEGA Genesis & Mega Drive Classics': 16, 'Grand Theft Auto IV': 17, 'Realm of the Mad God': 18, 'Marvel Heroes 2015': 19, 'Eldevin': 20, 'Dota 2': 21, 'BioShock': 22, 'Robocraft': 23, "Garry's Mod": 24, 'Jazzpunk': 25, 'Alan Wake': 26, 'BioShock 2': 27, 'Fallen Earth': 28, "Fallout New Vegas Courier's Stash": 29, 'Fallout New Vegas Dead Money': 30, 'Fallout New Vegas Honest Hearts': 31, 'Grand Theft Auto Episodes from Liberty City': 32, 'Hitman Absolution': 33, 'HuniePop Official Digital Art Collection': 34, 'HuniePop Original Soundtrack': 35, 'The Banner Saga - Mod Content': 36, 'The Elder Scrolls V Skyrim -

In [80]:
data['title_id'] = data['title']
data['title_id'].replace(to_replace = dic, inplace = True)
data.head()

,user_id,title,time,played,title_id
0,0,The Elder Scrolls V Skyrim,273.0,1.0,0
1,0,Fallout 4,0.0,0.0,1
2,0,Fallout 4,87.0,1.0,1
3,0,Spore,0.0,0.0,2
4,0,Spore,14.9,1.0,2


Para posteriormente conseguirmos resgatar os nomes dos jogos de modo a torná-los apresentáveis aos usuários, vamos salvar os nomes em um novo dataset temporário

In [81]:
game_names = data.drop(columns = ['user_id', 'time', 'played'])
data.drop(columns = ['title'], inplace = True)

E, para evitar duplicatas de título, os dados desse novo dataset serão sobrescritos com os dados específicos que realmente nos interessam, isto é, o nome de cada um dos jogos.

In [82]:
game_names = pd.Series(game_names['title'].unique())

In [83]:
game_names

0                The Elder Scrolls V Skyrim
1                                 Fallout 4
2                                     Spore
3                         Fallout New Vegas
4                             Left 4 Dead 2
                       ...                 
5150                     Warriors & Castles
5151    Romance of the Three Kingdoms Maker
5152                           Space Colony
5153                           Life is Hard
5154                      Executive Assault
Length: 5155, dtype: object

Então o nosso dataset original agora está assim:

In [84]:
data.head()

,user_id,time,played,title_id
0,0,273.0,1.0,0
1,0,0.0,0.0,1
2,0,87.0,1.0,1
3,0,0.0,0.0,2
4,0,14.9,1.0,2


In [85]:
users = pd.Series(data['user_id'].unique()).count()
titles = pd.Series(data['title_id'].unique()).count()

print(users, '\n', titles)

12393 
 5155


# Matriz de similaridade usuário-objeto

Para que seja possível relacionar cada usuário a um jogo, de modo a tornar possível avaliar a correlação entre diversos jogos e usuários, lançaremos mão de uma matriz de similaridade, que nada mais é do que uma matriz em que os jogos formam as colunas e os usuários as linhas, numa dada linha e coluna, correspondente a interação de um dado usuário com um certo jogo, será computado o valor do tempo que aquele usuário jogou aquele jogo. A partir dessas informações, utilizaremos um método de cálculo de similaridades para identificar comportamentos parecidos entre os usuários e recomendar um jogo a partir de outro dado como input.

In [86]:
data_matrix = np.zeros((users, titles))

for line in data.itertuples():
    data_matrix[line[1], line[4]] = line[2] #4 é o código do jogo


# Calculando similaridades

In [87]:
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import pairwise_distances

titles_similarity = pairwise_distances(data_matrix.T, metric = 'euclidean')

Para calcular a similaridade dentro da matriz, utilizaremos distâncias euclideanas como quantificador. Contudo, seria possível utilizar distância de Manhattan ou a similaridade de cossenos.

In [88]:
titles_similarity.shape

(5155, 5155)

# Recomendação!

Por fim, basta criar uma função que organize os resultados dessa matriz com base no objeto de interesse

In [89]:
def get_recommendation(title, metrica = titles_similarity):
    idx = dic[title] #Recebe o nome do jogo, que é convertido pelo dicionário para um title_id

    sim_scores = list(enumerate(metrica[idx])) #lista a coluna correspondente ao título passado

    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = False) #organiza de maneira ascendente a similaridade entre os diversos itens
    #itens mais parecidos têm distâncias menores entre si, logo aparecerão mais no início.

    #print(sim_scores)
    sim_scores = sim_scores[1:11] #filtra os 10 primeiros, excluindo o primeiro valor por ser, por essência, o próprio título buscado
        
    game_indices = [i[0] for i in sim_scores] #Salva os IDs de cada um dos jogos tidos como semelhantes

    games = []
    for i in sim_scores:
        games.append(game_names[i[0]]) #esses são convertidos de ID para os seus títulos usuais e adicionados em uma lista, que finalmente será apresentada

    return games

In [92]:
get_recommendation("Half-Life 2")

['Half-Life 2 Episode Two',
 'Half-Life 2 Episode One',
 'Half-Life Source',
 "Mirror's Edge",
 'Half-Life 2 Update',
 'Half-Life',
 'Synergy',
 'Shadowgrounds',
 'Shovel Knight',
 'Half-Life 2 Lost Coast']

In [91]:
get_recommendation("Dota 2")

['PAYDAY 2',
 'Path of Exile',
 'Dark Messiah of Might & Magic Multi-Player',
 'Fallout 4',
 'Borderlands',
 'Alan Wake',
 'Might & Magic Duel of Champions',
 'Company of Heroes Opposing Fronts',
 'Grand Theft Auto IV',
 'Rust']

Isso é tudo, pessoal 🫡

Bibliografia:

https://www.datacamp.com/tutorial/recommender-systems-python

https://medium.com/@lope.ai/recommendation-systems-from-scratch-in-python-pytholabs-6946491e76c2

https://webscope.sandbox.yahoo.com/catalog.php?datatype=r&guccounter=1&guce_referrer=aHR0cHM6Ly9naXRodWIuY29tL2Nhc2VyZWMvRGF0YXNldHMtZm9yLVJlY29tbWVuZGVyLVN5c3RlbXM&guce_referrer_sig=AQAAAAkyH74jyiIv4JxPjvejltL1_Sk-yDNtNAbIpHn2YfUnG1v-2mxj_XOD-qtpvdqg-aoNtTk9pzWVkYzz3ZbvN5C2_RrjVAowWPR7lmx-GidaMerX8qOzosJayRViVuW2IEoTjMAeZ8xJlIoK38-6GQAJOwZjFsSv0AyQNj4oagqX

https://en.wikipedia.org/wiki/Similarity_measure

https://realpython.com/build-recommendation-engine-collaborative-filtering/
